In [171]:
import requests
import pandas as pd

In [172]:
def get_message_from_tx(tx):
    m_list = []
    messages = tx['tx']['value']['msg']
    for m in messages:
        if 'execute_msg' in m['value'].keys():
            msg_obj = {
                'type':m['type'],
                'execute_msg':m['value']['execute_msg']
            }
            m_list.append(msg_obj)
    return m_list

In [173]:
def get_all_messages():
    loop = True
    m_list = []
    offset = 0
    while loop:
        data = requests.get(f"https://fcd.terra.dev/v1/txs?offset={offset}&limit=100&account=terra103z9cnqm8psy0nyxqtugg6m7xnwvlkqdzm4s4k")
        x = data.json()
        for tx in x['txs']:
            m_list = m_list + get_message_from_tx(tx)
        if 'next' in x.keys():
            offset = x['next']
        else:
            break
    m_list = [i for i in m_list if 'mint_nft' in i['execute_msg']]
    return m_list

In [174]:
def generate_dataframe(message_list):
    for i in message_list:
        i['token_id'] = i['execute_msg']['mint_nft']['token_id']
        i['name'] = i['execute_msg']['mint_nft']['extension']['name']
        i['image'] = i['execute_msg']['mint_nft']['extension']['image']
        i['attributes'] = i['execute_msg']['mint_nft']['extension']['attributes']
        i['description'] = i['execute_msg']['mint_nft']['extension']['description']
        i['token_uri'] = i['execute_msg']['mint_nft']['token_uri']
        
    for i in message_list:
        for a in i['attributes']:
            i[a['trait_type']] = a['value']
            
    df = pd.DataFrame(message_list)
    df['number_attributes'] = df.notnull()[
        ['backgrounds','suits','species','face','hair','glasses','headware','jewelry']
    ].sum(axis=1)
    df_lean = df[
        ['description','name','token_id','image','token_uri',
         'backgrounds','suits','species','face','hair','glasses',
         'headware','jewelry','number_attributes']
    ]
    return df_lean

In [175]:
messages = get_all_messages()
df = generate_dataframe(messages)
df.to_csv('gp.csv',index=None)